<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/Assessment_Mixtral_8x7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assessment on Mixtral 8x7B

* https://mistral.ai/news/mixtral-of-experts/
* https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1
* https://huggingface.co/mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bitgs8-metaoffload-HQQ
* https://huggingface.co/docs/transformers/v4.42.0/quantization/hqq

### Prerequisites
1. a Huggingface account and an
1. access tokens to put in below while logging in
1. patience: this will take 10-15 minutes to load libs and model and even generation will be really slow

### Why Mixtral
* explicitly tuned for European languages (like French, Italian, German and Spanish)
* performance close to GPT 3.5
* even quantized performance is good
* only uses fraction of parameters at a time
* theoretically able to unload parameters to CPU

### Comparing GPU microarchitectures
* T4/RTX 20: https://en.wikipedia.org/wiki/Turing_(microarchitecture)
  * V100 - professional variant of RTX 20 consumer line: https://en.wikipedia.org/wiki/Volta_(microarchitecture)
* A100/RTX 30: https://en.wikipedia.org/wiki/Ampere_(microarchitecture)
* L4/L40/RTX 40: https://en.wikipedia.org/wiki/Ada_Lovelace_(microarchitecture)
  * H100 - professional variant of RTX 40 consumer line, not available on Colab (yet?): https://en.wikipedia.org/wiki/Hopper_(microarchitecture)
* Future successor to both Hopper and Ada Lovelace: https://en.wikipedia.org/wiki/Blackwell_(microarchitecture)
* comparing GPUs: https://www.reddit.com/r/learnmachinelearning/comments/18gn1b2/choosing_the_right_gpu_for_your_workloads_a_dive/


In [1]:
%%time

# this will take a few minutes
!pip install hqq -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 107.2 MB/s eta 0:00:00
CPU times: user 1.08 s, sys: 161 ms, total: 1.24 s
Wall time: 3min 20s


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from IPython.display import Markdown

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in ca

In [5]:
!nvidia-smi

Wed Jul 17 17:19:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   31C    P8              16W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

***note:*** execute in a termial 'watch -n 0.5 nvidia-smi' to see the GPU usage and when the model is loaded onto it

In [6]:
%%time

import transformers
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
from hqq.core.quantize import *

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bitgs8-metaoffload-HQQ'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = HQQModelForCausalLM.from_quantized(model_id)

HQQLinear.set_backend(HQQBackend.ATEN_BACKPROP)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

qmodel.pt:   0%|          | 0.00/24.1G [00:00<?, ?B/s]

100%|██████████| 32/32 [00:11<00:00,  2.71it/s]

ATEN/CUDA backend not availabe. Make sure you install the hqq_aten library.
CPU times: user 1min 36s, sys: 58.8 s, total: 2min 34s
Wall time: 8min 11s


In [7]:
!nvidia-smi

Wed Jul 17 17:27:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   47C    P0              30W /  72W |  12997MiB / 23034MiB |     14%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
positive = [
  "With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.",
  "The socio-medical prerequisites for the prescribed aid supply have been met.",
  "Everyday relevant usage benefits have been determined.",
  "Socio-medical indication for the aid is confirmed.",
  "Contraindications have been excluded; there are no contraindications for the use of the requested aid."
]

In [9]:
negative = [
  "No specific findings can be derived from the diagnosis currently named as the basis for the regulation.",
  "According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.",
  "A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.",
  "From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.",
  "A medical justification for why a product not listed in the directory of aids should be used in the present case has not been transmitted."
]

In [10]:
# assessment = negative[0]
assessment = positive[0]

In [11]:
%%time

messages = [
    {"role": "user", "content": f'''
You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal.
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

# Assessment
{assessment}

'''},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=False
)
response = outputs[0][input_ids.shape[-1]:]
Markdown(tokenizer.decode(response, skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


CPU times: user 9min 20s, sys: 24.9 ms, total: 9min 20s
Wall time: 9min 19s


Yes, a positive recommendation for compensation is conceivable with the given diagnosis. The need for additional support to ensure the individual's basic needs are met can arise due to the limitations and challenges associated with the diagnosed health condition. This recommendation would be considered in the context of the individual's specific circumstances, including their ability to perform activities of daily living, their mobility, and their capacity for self-care. The primary goal of statutory health insurance is to ensure that individuals have access to the necessary healthcare services and resources to maintain their health and well-being, and this goal would guide the assessment process.

In [12]:
def eval_assessment(assessment):
  messages = [
    {"role": "user", "content": f'''
You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal.
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

# Assessment
{assessment}

'''},
]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=512,
      eos_token_id=terminators,
      pad_token_id=tokenizer.eos_token_id,
      do_sample=False
  )
  response = outputs[0][input_ids.shape[-1]:]
  result = tokenizer.decode(response, skip_special_tokens=True)
  if result.startswith("Yes"):
    return "Positive", result
  elif result.startswith("No"):
    return "Negative", result
  else:
    return "Neutral", result

## Negative

In [14]:
%%time

for assessment in negative:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: No specific findings can be derived from the diagnosis currently named as the basis for the regulation.
Negative: No, a positive or negative recommendation cannot be given based on the information provided. The diagnosis mentioned does not provide specific findings that can be used to make a definitive assessment. Further information and evaluation are needed to make a more informed recommendation.
-----
Assessment: According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.
Positive: Yes, a positive recommendation is given. The insured has already been provided with the functional product according to its area of application, as indicated by the service extracts from the health insurance. Therefore, it appears that the insured's needs have been met and the requested product is appropriate for their circumstances.
-----
Assessment: A medically comprehensible ex

## Positive

In [15]:
%%time

for assessment in positive:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.
Positive: Yes, a positive recommendation for compensation is conceivable with the given diagnosis. The need for additional support to ensure the individual's basic needs are met can arise due to the limitations and challenges associated with the diagnosed health condition. This recommendation would be considered in the context of the individual's specific circumstances, including their ability to perform activities of daily living, their mobility, and their capacity for self-care. The primary goal of statutory health insurance is to ensure that individuals have access to the necessary healthcare services and resources to maintain their health and well-being, and this goal would guide the assessment process.
-----
Assessment: The socio-medical prerequisites for the prescribed aid supply have been met.
Positive: Yes, based on the information provided, the socio-medical prerequisi

In [16]:
!nvidia-smi

Wed Jul 17 18:36:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   75C    P0              69W /  72W |  13519MiB / 23034MiB |     96%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--